<a href="https://colab.research.google.com/github/KenichiQaz/ML_Learning/blob/main/Own_Try10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from IPython.display import display,HTML

     |████████████████████████████████| 719 kB 7.2 MB/s 
     |████████████████████████████████| 1.3 MB 36.4 MB/s 
     |████████████████████████████████| 4.7 MB 36.8 MB/s 
     |████████████████████████████████| 365 kB 46.2 MB/s 
     |████████████████████████████████| 115 kB 57.8 MB/s 
     |████████████████████████████████| 120 kB 61.7 MB/s 
     |████████████████████████████████| 212 kB 47.4 MB/s 
     |████████████████████████████████| 127 kB 47.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
Mounted at /content/gdrive


In [2]:
# Word Tokenization with fastai
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
#get text files 
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
txt = files[0].open().read(); txt[:75]
# tokenize
spacy = WordTokenizer()
toks = first(spacy([txt]))
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#512) ['I','first','came','across',"'",'My','Tutor','Friend',"'",'accidentally','one','or','two','years','ago','while','TV','surfing','.','Prior','to','that',',','I',"'d",'never','watched','any','Korean','films'...]


# Subword tokenization


In [4]:
txts = L(o.open().read() for o in files[:2000])
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [5]:
# Numericalization with fastai
tkn = Tokenizer(spacy)
toks200 = txts[:200].map(tkn)
num = Numericalize()
num.setup(toks200)
nums = num(toks)[:20]; nums

TensorText([  0, 116, 296, 805,  53,   0,   0,   0,  53,   0,  46,  62, 107, 162, 508, 138,   0,   0,  11,   0])

# Preping text for NLP model training

In [6]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [7]:
# Batch 1
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [8]:
# Batch 2
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [9]:
# Batch 3
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [10]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)
x,y = first(dl)
x.shape,y.shape
' '.join(num.vocab[o] for o in x[0][:20])
' '.join(num.vocab[o] for o in y[0][:20])

"i first came across ' my xxmaj xxunk xxmaj friend ' xxunk one or two years ago while xxup tv"

In [11]:
# Text Classifier
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this movie is bad , so bad that my mother who can barely stand the "" suspense "" of xxmaj disney 's xxmaj snow xxmaj white , was chuckling through out the entire movie . xxmaj my first warning should 've been that it was in the $ 5.50 bin at xxmaj wal - mart . xxmaj but i have actually found some good movies in that bin , so i ca n't fault if for this debacle","xxmaj this movie is bad , so bad that my mother who can barely stand the "" suspense "" of xxmaj disney 's xxmaj snow xxmaj white , was chuckling through out the entire movie . xxmaj my first warning should 've been that it was in the $ 5.50 bin at xxmaj wal - mart . xxmaj but i have actually found some good movies in that bin , so i ca n't fault if for this debacle ."
1,"in the movie . \n\n xxmaj however , the writing and the characterizations were just dreck , the worst of xxup tv sitcoms rolled up in a film canister . xxmaj all of the supporting characters were one - dimensional ( if there are n't any lower options ) , and only seemed somewhat familiar with human interaction on xxmaj planet xxmaj earth . xxmaj as a result , situations that the writers must have thought hilarious turned out to","the movie . \n\n xxmaj however , the writing and the characterizations were just dreck , the worst of xxup tv sitcoms rolled up in a film canister . xxmaj all of the supporting characters were one - dimensional ( if there are n't any lower options ) , and only seemed somewhat familiar with human interaction on xxmaj planet xxmaj earth . xxmaj as a result , situations that the writers must have thought hilarious turned out to be"


In [12]:
# Fine-Tuning the Language Model
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()
learn.fit_one_cycle(1, 2e-2)
# saving
learn.save('1epoch')
#continue training
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)
# save the final encoder
learn.save_encoder('finetuned')

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: ignored

In [ ]:
# text generation test
TEXT = "I didn't like this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))